# Imports

In [1]:
import pandas as pd
import numpy as np

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import pylab as plt

In [3]:
import statsmodels.api as sm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
from statsmodels.tsa.ar_model import AR

In [5]:
from sklearn.metrics import mean_squared_error as mse

# Functions

In [6]:
def clean_data():
    
    global db1
    db1 = db1[(db1.num_infections > 0)].reset_index()
    db1 = db1.drop(columns = ['index'], axis = 1)
    
    db1['date'] = pd.to_datetime(db1.date)
    
   
    db1['autonomous_region'] = db1.autonomous_region.apply(lambda x: 'Comunidad Valenciana' if x == 'Valenciana, Comunidad' else x)
    db1['autonomous_region'] = db1.autonomous_region.apply(lambda x: 'Comunidad de Madrid' if x == 'Madrid, Comunidad de' else x)
    db1['autonomous_region'] = db1.autonomous_region.apply(lambda x: 'Región de de Murcia' if x == 'Murcia, Región de' else x)
    db1['autonomous_region'] = db1.autonomous_region.apply(lambda x: 'Comunidad Foral de Navarra' if x == 'Navarra, Comunidad Foral de' else x)
    db1['autonomous_region'] = db1.autonomous_region.apply(lambda x: 'Principado de Asturias' if x == 'Asturias, Principado de' else x)
    
    
    db1['province'] = db1.province.apply(lambda x: 'Alicante' if x == 'Alicante/Alacant' else x)
    db1['province'] = db1.province.apply(lambda x: 'Castellón' if x == 'Castellón/Castelló' else x)
    db1['province'] = db1.province.apply(lambda x: 'Araba' if x == 'Araba/Álava' else x)
    
    db1['sex'] = db1.sex.apply(lambda x: 'UKNOWN' if x == 'NC' else x)
    
    db1['age_interval'] = db1.age_interval.apply(lambda x: 'UKNOWN' if x == 'NC' else x)

In [7]:
def cumulative(dataframe):
    
    cumu_num_infections = dataframe.num_infections.cumsum()
    cumu_num_hosp = dataframe.num_hosp.cumsum()
    cumu_num_uci = dataframe.num_uci.cumsum()
    cumu_num_dead = dataframe.num_dead.cumsum()

    cumulative = pd.DataFrame({'cumu_num_infections': cumu_num_infections, 'cumu_num_hosp': cumu_num_hosp, 
                               'cumu_num_uci': cumu_num_uci, 'cumu_num_dead': cumu_num_dead})
    dataframe = pd.concat([dataframe, cumulative], axis = 1)
    return dataframe

In [8]:
def freq_rel(dataframe):
    
    dataframe['Number of infection (%)'] = [round(i/ dataframe['num_infections'].sum(),3)*100 for i in dataframe['num_infections']]
    dataframe['Number of hospitalisation (%)'] = [round(i/ dataframe['num_hosp'].sum(),3)*100 for i in dataframe['num_hosp']]
    dataframe['Number of Intensive Care Unit (%)'] = [round(i/ dataframe['num_uci'].sum(),3)*100 for i in dataframe['num_uci']]
    dataframe['Number of deaths (%)'] = [round(i/ dataframe['num_dead'].sum(),3)*100 for i in dataframe['num_dead']]

In [9]:
def organise(column):
    
    name = db1.groupby(column).sum()
    freq_rel(name)
    
    return name

# Data base

In [10]:
db1 =pd.read_csv('/content/covid-19-sample_old.csv') 

In [11]:
clean_data()

In [12]:
db1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55471 entries, 0 to 55470
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   date               55471 non-null  datetime64[ns]
 1   autonomous_region  55471 non-null  object        
 2   province           55471 non-null  object        
 3   sex                55471 non-null  object        
 4   age_interval       55471 non-null  object        
 5   num_infections     55471 non-null  int64         
 6   num_hosp           55471 non-null  int64         
 7   num_uci            55471 non-null  int64         
 8   num_dead           55471 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(3), object(4)
memory usage: 3.8+ MB


In [13]:
bydate = organise('date')

In [14]:
bydate


,num_infections,num_hosp,num_uci,num_dead,Number of infection (%),Number of hospitalisation (%),Number of Intensive Care Unit (%),Number of deaths (%)
date,,,,,,,,
2020-02-12,1,0,0,0.0,0.0,0.0,0.0,0.0
2020-02-21,1,0,0,0.0,0.0,0.0,0.0,0.0
2020-02-22,1,0,0,0.0,0.0,0.0,0.0,0.0
2020-02-27,4,1,0,0.0,0.0,0.0,0.0,0.0
2020-02-28,1,0,0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2021-06-04,841,29,3,2.0,0.1,0.0,0.0,0.0
2021-06-05,461,19,4,3.0,0.0,0.0,0.1,0.0
2021-06-06,358,20,1,0.0,0.0,0.0,0.0,0.0


In [15]:
infections = pd.DataFrame(bydate.num_infections, columns = ['num_infections'])

In [16]:
infections

,num_infections
date,
2020-02-12,1
2020-02-21,1
2020-02-22,1
2020-02-27,4
2020-02-28,1
...,...
2021-06-04,841
2021-06-05,461
2021-06-06,358


# ARMA

In [19]:
from statsmodels.tsa.arima_model import ARMA

## Number of infections

In [18]:
import warnings
warnings.simplefilter('ignore')
res = []
for i in range(1, 20):
    for j in range(1,10): 
        for k in range(1,7):
            train, test = infections[:-i], infections[-i:]
            model=ARMA(train, order=(j, k)).fit(disp=False)
            pred=model.predict(len(train), len(infections)-1)
            test['predictions'] = list(pred)
            test['diff'] = abs(test['num_infections'] - test['predictions'])
            res.append(((i, j, k), mse(test['num_infections'], pred)**0.5))
            #print(mse(test['num_infections'], pred)**0.5, '\nerror mae:', test['diff'].sum()/len(test))

ValueError: ignored

In [ ]:
errors = []
for i, e in enumerate(res):
    errors.append(res[i][1])

sorted(errors)


In [ ]:
best = []
for j, k in enumerate(errors):
    if k == min(errors):
        best.append(res[j])
best

## Number of hospitalisation

In [ ]:
hosp = pd.DataFrame(bydate.num_hosp, columns = ['num_hosp'])

In [ ]:
hosp

In [ ]:
res = []
for i in range(1, 20):
    for j in range(1,10): 
        for k in range(1,7):
            train, test = hosp[:-i], hosp[-i:]
            model=ARMA(train, order=(j, k)).fit(disp=False)
            pred=model.predict(len(train), len(hosp)-1)
            res.append(((i, j, k), mse(test['num_hosp'], pred)**0.5))

In [ ]:
errors = []
for i, e in enumerate(res):
    errors.append(res[i][1])

In [ ]:
best = []
for j, k in enumerate(errors):
    if k == min(errors):
        best.append(res[j])
best

## Number of Intensive Care Unites

In [ ]:
uci = pd.DataFrame(bydate.num_uci, columns = ['num_uci'])

In [ ]:
uci

In [ ]:
res = []
for i in range(1, 20):
    for j in range(1,10): 
        for k in range(1,7):
            train, test = uci[:-i], uci[-i:]
            model=ARMA(train, order=(j, k)).fit(disp=False)
            pred=model.predict(len(train), len(uci)-1)
            res.append(((i, j, k), mse(test['num_uci'], pred)**0.5))
            #print(mse(test['num_infections'], pred)**0.5, '\nerror mae:', test['diff'].sum()/len(test))

In [ ]:
errors = []
for i, e in enumerate(res):
    errors.append(res[i][1])
sorted(errors)

In [ ]:
best = []
for j, k in enumerate(errors):
    if k == min(errors):
        best.append(res[j])
best

## Number of deaths

In [ ]:
dead = pd.DataFrame(bydate.num_dead, columns = ['num_dead'])

In [ ]:
res = []
for i in range(1, 20):
    for j in range(1,10): 
        for k in range(1,7):
            train, test = dead[:-i], dead[-i:]
            model=ARMA(train, order=(j, k)).fit(disp=False)
            pred=model.predict(len(train), len(infections)-1)
            res.append(((i, j, k), mse(test['num_dead'], pred)**0.5))
            #print(mse(test['num_infections'], pred)**0.5, '\nerror mae:', test['diff'].sum()/len(test))

In [ ]:
errors = []
for i, e in enumerate(res):
    errors.append(res[i][1])

In [ ]:
best = []
for j, k in enumerate(errors):
    if k == min(errors):
        best.append(res[j])
best

# FB Prophet

In [ ]:
from fbprophet import Prophet
plt.style.use('fivethirtyeight')

In [ ]:
def time_features(df):
    
    df=df.copy()
    
    df['date']=df.index
    
    df['hour']=df.date.dt.hour
    df['day_of_week']=df.date.dt.dayofweek
    df['quarter']=df.date.dt.quarter
    df['month']=df.date.dt.month
    df['year']=df.date.dt.year
    df['day_of_year']=df.date.dt.dayofyear
    df['day_of_month']=df.date.dt.day
    df['week_of_year']=df.date.dt.weekofyear
    
    return df

In [ ]:
train, test = bydate.num_infections[:-45], bydate.num_infections[-45:]
train=train.reset_index().rename(columns={'date':'ds', 'num_infections':'y'})
test=test.reset_index().rename(columns={'date':'ds', 'num_infections':'y'})

In [ ]:
%%time

model=Prophet().fit(train)

pred=model.predict(test)

In [ ]:
f, ax=plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(15)
fig=model.plot(pred, ax=ax)

In [ ]:
f, ax=plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(15)
ax.scatter(test.ds, test.y, color='r')
fig=model.plot(pred, ax=ax)

In [ ]:
mse(test.y, pred.yhat)**0.5

In [ ]:
out = []
for i in range(1, 100):
  train, test = bydate.num_infections[:-i], bydate.num_infections[-i:]
  train=train.reset_index().rename(columns={'date':'ds', 'num_infections':'y'})
  test=test.reset_index().rename(columns={'date':'ds', 'num_infections':'y'})
  model=Prophet().fit(train)
  pred=model.predict(test)
  out.append((i, mse(test.y, pred.yhat)**0.5))

In [ ]:
sorted(out)

In [ ]:
! pip install h2o

In [ ]:
import h2o
from h2o.automl import H2OAutoML

In [ ]:
h2o.init()

In [ ]:
infections = infections.reset_index()

In [ ]:
infections = infections.to_csv('infections.csv', index=False)

In [ ]:
df=h2o.import_file('/content/infections.csv')

In [ ]:
df

In [ ]:
X=df.columns
y='num_infections'
X.remove(y)

In [ ]:
df[y]=df[y].asfactor()

In [ ]:
aml=H2OAutoML(max_models=3, seed=1)

aml.train(x=X, y=y, training_frame=df)

In [ ]:
lb=aml.leaderboard

lb.head(rows=lb.nrows)

In [ ]:
aml.leader

In [ ]:
pred=aml.leader.predict(X)

pred

In [ ]:
y

In [ ]:
X